In [108]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import log_loss
from tqdm import tqdm


In [109]:
train = pd.read_pickle('train_6model_fasterfeats.pickle')
test_list = pd.read_pickle('test_6models_fasterfeats.pickle')


In [110]:
target_cols = ['any', 'epidural', 'subdural', 'subarachnoid', 'intraventricular', 'intraparenchymal']
X_cols = train.columns.drop(['StudyInstanceUID', 'folds', 'ID'] + target_cols).tolist()

In [111]:
_mean = train[X_cols].mean()

In [112]:
train[X_cols] = train[X_cols].fillna(_mean)

In [124]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
X_train = []
X_valid = []
y_train = []
y_valid = []
std = MinMaxScaler()
std.fit(train.drop(['ID', 'StudyInstanceUID', 'folds'] + target_cols, axis=1).values)
for i in range(5):
    X_train.append(
        std.transform(train.query('folds!=@i').drop(['ID', 'StudyInstanceUID', 'folds'] + target_cols, axis=1).values)
    )
    X_valid.append(
        std.transform(train.query('folds==@i').drop(['ID', 'StudyInstanceUID', 'folds'] + target_cols, axis=1).values)
    )
    y_train.append(train.query('folds!=@i')[target_cols].values)
    y_valid.append(train.query('folds==@i')[target_cols].values)

In [125]:
# X_train = np.array(X_train)#.transpose(1, 0, 2)[:, np.newaxis, ...]
# X_valid = np.array(X_valid)#.transpose(1, 0, 2)[:, np.newaxis, ...]

In [126]:
class StackingDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        return torch.FloatTensor(self.X[idx]), torch.FloatTensor(self.y[idx])

In [127]:
for X_train_, X_valid_, y_train_, y_valid_ in zip(X_train, X_valid, y_train, y_valid):
    train_loader = DataLoader(
        StackingDataset(X_train_, y_train_),
        shuffle=True,
        batch_size=2048,
        num_workers=4,
        pin_memory=True,
    )
    valid_loader = DataLoader(
        StackingDataset(X_valid_, y_valid_),
        batch_size=2048,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
    )
    break

In [130]:
class MLPNet (nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(len(X_cols), 100)
        self.bn1 = nn.BatchNorm1d(100)
        self.fc2 = nn.Linear(100, 200)
        self.fc3 = nn.Linear(200, 6)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MLPNet().to(device)

# optimizing
criterion = nn.BCEWithLogitsLoss(weight=torch.FloatTensor([2, 1, 1, 1, 1, 1]).cuda())
optimizer = torch.optim.Adam(model.parameters(), 5e-3)


In [131]:
for epoch in range(50):
    losses = []
    for x, y in train_loader:
        pred = model(x.cuda())
        loss = criterion(pred, y.cuda())
        if not np.isnan(loss.item()):
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            losses.append(loss.item())
    losses_valid = []
    with torch.no_grad():
        model.eval()
        for x, y in valid_loader:
            pred = model(x.cuda())
            loss = criterion(pred, y.cuda())
            losses_valid.append(loss.item())
    print(f'Epoch[{epoch}]', 'Train:', np.mean(losses), 'Valid:', np.mean(losses_valid))
    model.train()

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
    w.join()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 138, in join
  File "/usr/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: ca

Epoch[0] Train: 0.7872190470878895 Valid: 0.7876971445300363


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Traceback (most recent call last):
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    self._shutdown_workers()
    w.join()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
  File "/usr/lib/python3.7/multiprocessing/process.py", line 138, in join
    w.join()
    assert self._parent_pid == os.getpid(), 'can only join a child process'
  File "/usr/lib/python3.7/multiprocessing/process.py", line 138, in join
AssertionError: ca

Epoch[1] Train: 0.7864798094217594 Valid: 0.7875616270484347


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
    self._shutdown_workers()
    self.

...

Epoch[10] Train: 0.785721594782976 Valid: 0.7873302345926111


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
    w.join()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 138, in join
  File "/usr/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
    assert self._p

Epoch[11] Train: 0.7856833985218635 Valid: 0.7879985092264233


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff1d211bf80>
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
    self._shutdown_workers()
    self.

KeyboardInterrupt: 

torch.Size([512, 576])